In [14]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

import os
from PIL import Image
import random


In [2]:
def resize_image(image):
    IMG_SIZE = 224
    image = tf.cast(image, tf.float32)
    image = (image/255)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image

In [5]:
full_data = []
combined_data = 'datasets/Combined Data'
three_classes = 'datasets/Tube_Effusion_Normal - 11_7_19'

data_path = three_classes
class_names = [x for x in os.listdir(data_path) if os.path.isdir(data_path + '/' + x)]
print(class_names)
class_dict = {}
for i, name in enumerate(class_names):
    class_dict[name] = i
print(class_dict)

def get_data(): 
    path = data_path
    for di in os.listdir(path):
        dirpath = path+"/"+di
        if (os.path.isdir(dirpath)):

            image_files = [x for x in os.listdir(dirpath) if (x[-4:] == "jpeg" or x[-3:] == "png" or x[-3:] == 'jpg')]

            for img in image_files:
                image = Image.open(dirpath+"/"+img)
                data = np.asarray(image)
                data = resize_image(data)
                image_tuple = np.array([data, class_dict[di]])
                full_data.append(image_tuple)


get_data()
full_data = np.array(full_data)

random.shuffle(full_data)

full_data.shape

['Tube', 'Effusion', 'Normal']
{'Tube': 0, 'Effusion': 1, 'Normal': 2}


/var/folders/84/c6fx1p654h5cgl4xrm3kgys80000gn/T/ipykernel_16033/2941472138.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  image_tuple = np.array([data, class_dict[di]])


(454, 2)

In [6]:
X_full = np.array([x[0] for x in full_data])
y_full = np.array([x[1] for x in full_data])

print(X_full[0].shape)

(224, 224, 3)


In [7]:
split = int(0.7*len(X_full))
X_train = X_full[:split]
X_test = X_full[split:]

y_train = y_full[:split]
y_test = y_full[split:]



In [42]:
from keras_cv_attention_models import beit
# Will download and load pretrained imagenet21k-ft1k weights.
mm = beit.BeitBasePatch16(input_shape=(224, 224, 3), pretrained="imagenet21k-ft1k") 

346857472/346843944 [==============================] - 55s 0us/step
>>>> Load pretrained from: /Users/dhruv/.keras/models/beit_base_patch16_224_imagenet21k-ft1k.h5


In [9]:
mm.summary()

Model: "beit_large_patch16"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 14, 14, 1024  787456      ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 reshape (Reshape)              (None, 196, 1024)    0           ['stem_conv[0][0]']              
                                                                                 

                                                                                                  
 block0_mlp_dense_2 (Dense)     (None, 197, 1024)    4195328     ['block0_mlp_gelugelu[0][0]']    
                                                                                                  
 block0_mlp_gamma (ChannelAffin  (None, 197, 1024)   1024        ['block0_mlp_dense_2[0][0]']     
 e)                                                                                               
                                                                                                  
 block0_mlp_output (Add)        (None, 197, 1024)    0           ['block0_attn_out[0][0]',        
                                                                  'block0_mlp_gamma[0][0]']       
                                                                                                  
 block1_attn_ln (LayerNormaliza  (None, 197, 1024)   2048        ['block0_mlp_output[0][0]']      
 tion)    

 block1_mlp_gamma (ChannelAffin  (None, 197, 1024)   1024        ['block1_mlp_dense_2[0][0]']     
 e)                                                                                               
                                                                                                  
 block1_mlp_output (Add)        (None, 197, 1024)    0           ['block1_attn_out[0][0]',        
                                                                  'block1_mlp_gamma[0][0]']       
                                                                                                  
 block2_attn_ln (LayerNormaliza  (None, 197, 1024)   2048        ['block1_mlp_output[0][0]']      
 tion)                                                                                            
                                                                                                  
 block2_attn_qkv (Dense)        (None, 197, 3072)    3145728     ['block2_attn_ln[0][0]']         
          

 block2_mlp_output (Add)        (None, 197, 1024)    0           ['block2_attn_out[0][0]',        
                                                                  'block2_mlp_gamma[0][0]']       
                                                                                                  
 block3_attn_ln (LayerNormaliza  (None, 197, 1024)   2048        ['block2_mlp_output[0][0]']      
 tion)                                                                                            
                                                                                                  
 block3_attn_qkv (Dense)        (None, 197, 3072)    3145728     ['block3_attn_ln[0][0]']         
                                                                                                  
 tf.reshape_15 (TFOpLambda)     (None, 197, 3072)    0           ['block3_attn_qkv[0][0]']        
                                                                                                  
 tf.split_

 block4_attn_ln (LayerNormaliza  (None, 197, 1024)   2048        ['block3_mlp_output[0][0]']      
 tion)                                                                                            
                                                                                                  
 block4_attn_qkv (Dense)        (None, 197, 3072)    3145728     ['block4_attn_ln[0][0]']         
                                                                                                  
 tf.reshape_20 (TFOpLambda)     (None, 197, 3072)    0           ['block4_attn_qkv[0][0]']        
                                                                                                  
 tf.split_4 (TFOpLambda)        [(None, 197, 1024),  0           ['tf.reshape_20[0][0]']          
                                 (None, 197, 1024),                                               
                                 (None, 197, 1024)]                                               
          

...

 lambda_39 (Lambda)             (None, 16, 197, 64)  0           ['block19_attn_attention_scores[0
                                                                 ][0]',                           
                                                                  'tf.compat.v1.transpose_78[0][0]
                                                                 ']                               
                                                                                                  
 tf.compat.v1.transpose_79 (TFO  (None, 197, 16, 64)  0          ['lambda_39[0][0]']              
 pLambda)                                                                                         
                                                                                                  
 tf.reshape_99 (TFOpLambda)     (None, 197, 1024)    0           ['tf.compat.v1.transpose_79[0][0]
                                                                 ']                               
          

                                                                 ']                               
                                                                                                  
 tf.compat.v1.transpose_83 (TFO  (None, 197, 16, 64)  0          ['lambda_41[0][0]']              
 pLambda)                                                                                         
                                                                                                  
 tf.reshape_104 (TFOpLambda)    (None, 197, 1024)    0           ['tf.compat.v1.transpose_83[0][0]
                                                                 ']                               
                                                                                                  
 block20_attn_output (Dense)    (None, 197, 1024)    1049600     ['tf.reshape_104[0][0]']         
                                                                                                  
 block20_a

 pLambda)                                                                                         
                                                                                                  
 tf.reshape_109 (TFOpLambda)    (None, 197, 1024)    0           ['tf.compat.v1.transpose_87[0][0]
                                                                 ']                               
                                                                                                  
 block21_attn_output (Dense)    (None, 197, 1024)    1049600     ['tf.reshape_109[0][0]']         
                                                                                                  
 block21_attn_gamma (ChannelAff  (None, 197, 1024)   1024        ['block21_attn_output[0][0]']    
 ine)                                                                                             
                                                                                                  
 block21_a

                                                                 ']                               
                                                                                                  
 block22_attn_output (Dense)    (None, 197, 1024)    1049600     ['tf.reshape_114[0][0]']         
                                                                                                  
 block22_attn_gamma (ChannelAff  (None, 197, 1024)   1024        ['block22_attn_output[0][0]']    
 ine)                                                                                             
                                                                                                  
 block22_attn_out (Add)         (None, 197, 1024)    0           ['block21_mlp_output[0][0]',     
                                                                  'block22_attn_gamma[0][0]']     
                                                                                                  
 block22_m

                                                                                                  
 block23_attn_gamma (ChannelAff  (None, 197, 1024)   1024        ['block23_attn_output[0][0]']    
 ine)                                                                                             
                                                                                                  
 block23_attn_out (Add)         (None, 197, 1024)    0           ['block22_mlp_output[0][0]',     
                                                                  'block23_attn_gamma[0][0]']     
                                                                                                  
 block23_mlp_ln (LayerNormaliza  (None, 197, 1024)   2048        ['block23_attn_out[0][0]']       
 tion)                                                                                            
                                                                                                  
 block23_m

In [39]:
x = mm.layers[-2].output
print(x)
output = keras.layers.Dense(units=len(class_names), activation="softmax")(x)

model = keras.models.Model(inputs=mm.input, outputs=output)
for layer in model.layers: layer.trainable = True
for layer in model.layers[:-10]: layer.trainable = False

model.summary()


KerasTensor(type_spec=TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None), name='out_ln/batchnorm/add_1:0', description="created by layer 'out_ln'")
Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 14, 14, 1024  787456      ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 reshape (Reshape)              (

 block0_mlp_gelugelu (Activatio  (None, 197, 4096)   0           ['block0_mlp_dense_1[0][0]']     
 n)                                                                                               
                                                                                                  
 block0_mlp_dense_2 (Dense)     (None, 197, 1024)    4195328     ['block0_mlp_gelugelu[0][0]']    
                                                                                                  
 block0_mlp_gamma (ChannelAffin  (None, 197, 1024)   1024        ['block0_mlp_dense_2[0][0]']     
 e)                                                                                               
                                                                                                  
 block0_mlp_output (Add)        (None, 197, 1024)    0           ['block0_attn_out[0][0]',        
                                                                  'block0_mlp_gamma[0][0]']       
          

 block1_mlp_dense_2 (Dense)     (None, 197, 1024)    4195328     ['block1_mlp_gelugelu[0][0]']    
                                                                                                  
 block1_mlp_gamma (ChannelAffin  (None, 197, 1024)   1024        ['block1_mlp_dense_2[0][0]']     
 e)                                                                                               
                                                                                                  
 block1_mlp_output (Add)        (None, 197, 1024)    0           ['block1_attn_out[0][0]',        
                                                                  'block1_mlp_gamma[0][0]']       
                                                                                                  
 block2_attn_ln (LayerNormaliza  (None, 197, 1024)   2048        ['block1_mlp_output[0][0]']      
 tion)                                                                                            
          

 e)                                                                                               
                                                                                                  
 block2_mlp_output (Add)        (None, 197, 1024)    0           ['block2_attn_out[0][0]',        
                                                                  'block2_mlp_gamma[0][0]']       
                                                                                                  
 block3_attn_ln (LayerNormaliza  (None, 197, 1024)   2048        ['block2_mlp_output[0][0]']      
 tion)                                                                                            
                                                                                                  
 block3_attn_qkv (Dense)        (None, 197, 3072)    3145728     ['block3_attn_ln[0][0]']         
                                                                                                  
 tf.reshap

                                                                  'block3_mlp_gamma[0][0]']       
                                                                                                  
 block4_attn_ln (LayerNormaliza  (None, 197, 1024)   2048        ['block3_mlp_output[0][0]']      
 tion)                                                                                            
                                                                                                  
 block4_attn_qkv (Dense)        (None, 197, 3072)    3145728     ['block4_attn_ln[0][0]']         
                                                                                                  
 tf.reshape_20 (TFOpLambda)     (None, 197, 3072)    0           ['block4_attn_qkv[0][0]']        
                                                                                                  
 tf.split_4 (TFOpLambda)        [(None, 197, 1024),  0           ['tf.reshape_20[0][0]']          
          

...

 pLambda)                                                                                         
                                                                                                  
 lambda_39 (Lambda)             (None, 16, 197, 64)  0           ['block19_attn_attention_scores[0
                                                                 ][0]',                           
                                                                  'tf.compat.v1.transpose_78[0][0]
                                                                 ']                               
                                                                                                  
 tf.compat.v1.transpose_79 (TFO  (None, 197, 16, 64)  0          ['lambda_39[0][0]']              
 pLambda)                                                                                         
                                                                                                  
 tf.reshap

                                                                 ][0]',                           
                                                                  'tf.compat.v1.transpose_82[0][0]
                                                                 ']                               
                                                                                                  
 tf.compat.v1.transpose_83 (TFO  (None, 197, 16, 64)  0          ['lambda_41[0][0]']              
 pLambda)                                                                                         
                                                                                                  
 tf.reshape_104 (TFOpLambda)    (None, 197, 1024)    0           ['tf.compat.v1.transpose_83[0][0]
                                                                 ']                               
                                                                                                  
 block20_a

                                                                                                  
 tf.compat.v1.transpose_87 (TFO  (None, 197, 16, 64)  0          ['lambda_43[0][0]']              
 pLambda)                                                                                         
                                                                                                  
 tf.reshape_109 (TFOpLambda)    (None, 197, 1024)    0           ['tf.compat.v1.transpose_87[0][0]
                                                                 ']                               
                                                                                                  
 block21_attn_output (Dense)    (None, 197, 1024)    1049600     ['tf.reshape_109[0][0]']         
                                                                                                  
 block21_attn_gamma (ChannelAff  (None, 197, 1024)   1024        ['block21_attn_output[0][0]']    
 ine)     

                                                                                                  
 tf.reshape_114 (TFOpLambda)    (None, 197, 1024)    0           ['tf.compat.v1.transpose_91[0][0]
                                                                 ']                               
                                                                                                  
 block22_attn_output (Dense)    (None, 197, 1024)    1049600     ['tf.reshape_114[0][0]']         
                                                                                                  
 block22_attn_gamma (ChannelAff  (None, 197, 1024)   1024        ['block22_attn_output[0][0]']    
 ine)                                                                                             
                                                                                                  
 block22_attn_out (Add)         (None, 197, 1024)    0           ['block21_mlp_output[0][0]',     
          

                                                                                                  
 block23_attn_output (Dense)    (None, 197, 1024)    1049600     ['tf.reshape_119[0][0]']         
                                                                                                  
 block23_attn_gamma (ChannelAff  (None, 197, 1024)   1024        ['block23_attn_output[0][0]']    
 ine)                                                                                             
                                                                                                  
 block23_attn_out (Add)         (None, 197, 1024)    0           ['block22_mlp_output[0][0]',     
                                                                  'block23_attn_gamma[0][0]']     
                                                                                                  
 block23_mlp_ln (LayerNormaliza  (None, 197, 1024)   2048        ['block23_attn_out[0][0]']       
 tion)    

In [41]:
model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=4, 
                    validation_data=(X_test, y_test))

Epoch 1/4


2022-06-30 15:55:02.230203: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - ETA: 0s - loss: 0.6643 - accuracy: 0.6656

2022-06-30 15:56:02.581469: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 103s 9s/step - loss: 0.6643 - accuracy: 0.6656 - val_loss: 2.2142 - val_accuracy: 0.2920
Epoch 2/4
10/10 [==============================] - 69s 7s/step - loss: 0.6500 - accuracy: 0.6372 - val_loss: 2.2119 - val_accuracy: 0.2920
Epoch 3/4
10/10 [==============================] - 75s 8s/step - loss: 0.6544 - accuracy: 0.6341 - val_loss: 2.2651 - val_accuracy: 0.2920
Epoch 4/4
10/10 [==============================] - 85s 9s/step - loss: 0.6491 - accuracy: 0.6782 - val_loss: 2.1953 - val_accuracy: 0.2920


In [43]:
COLOR = 'black'
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR

def predict(model, image, correct_label):
    prediction = model.predict(np.array([image]))
    predicted_class = class_names[np.argmax(prediction)]

    show_image(image, class_names[correct_label], predicted_class)

def show_image(img, label, guess):
    plt.figure()
    plt.imshow(img, cmap=plt.cm.binary)
    plt.title("Excpected: " + label)
    plt.xlabel("Guess: " + guess)
    plt.colorbar()
    plt.grid(False)
    plt.show()

def get_number():
    while True:
        num = input("Pick a number: ")
    if num.isdigit():
        num = int(num)
        if 0 <= num <= 1000:
            return int(num)
    else:
        print("Try again...")


for num in range(0, 10):
    image = X_test[num]
    label = y_test[num]
    predict(mm, image, label)

2022-06-30 16:09:34.201230: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


IndexError: list index out of range

In [ ]:
model = keras.Sequential()
a = layers.Dense(2, activation="relu")
display(a)
model.add(a)
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))